In [1]:
#Celdas de importaciones
import pandas as pd
from fastText import train_supervised
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk.corpus import stopwords

In [21]:
#Seteo de variables globales
ds_path='./Dataset/Comentarios para clasificar.xls'
trainPath='./Resources/train.txt'
testPath='./Resources/test.txt'
outputPath='./Resources/output.txt'

In [3]:
#Cargamos el Dataset
df=pd.read_excel(ds_path)
df

,Comentario,Clasficacion
0,‼️Del 10 al 20 de Septiembre‼️\nDisfruta del B...,NO
1,"""AHORA SI GUERRA CIVIL""!\nPor Favor oremos por...",NO
2,"""L' qui mi estudio di 20 anni non realiza lo v...",NO
3,##La población Boliviana denuncia los dieferen...,NO
4,#AberturasAlCosto.\nDirección Perú esquina Cos...,NO
...,...,...
1299,venta,SI
1300,Vi esta pilar centro muy grande',SI
1301,X favor busco casa o departamento para arquila...,SI
1302,"Ya está alquilado?', 'Esta alquilado???'",SI


In [4]:
#Para trabajar mas comodo voy a convertir las palabras clasificatorias NO y SI en 0 y 1 respectivaente
df=df.replace({'Clasficacion': {'NO': 0, 'SI': 1}})
df

,Comentario,Clasficacion
0,‼️Del 10 al 20 de Septiembre‼️\nDisfruta del B...,0
1,"""AHORA SI GUERRA CIVIL""!\nPor Favor oremos por...",0
2,"""L' qui mi estudio di 20 anni non realiza lo v...",0
3,##La población Boliviana denuncia los dieferen...,0
4,#AberturasAlCosto.\nDirección Perú esquina Cos...,0
...,...,...
1299,venta,1
1300,Vi esta pilar centro muy grande',1
1301,X favor busco casa o departamento para arquila...,1
1302,"Ya está alquilado?', 'Esta alquilado???'",1


In [5]:
#Contamos la cantidad de veces en que se repiten las clasificaciones para determinar cuando desbalanceado esta el dataset
df['Clasficacion'].value_counts() 

0    1018
1     286
Name: Clasficacion, dtype: int64

In [6]:
df

,Comentario,Clasficacion
0,‼️Del 10 al 20 de Septiembre‼️\nDisfruta del B...,0
1,"""AHORA SI GUERRA CIVIL""!\nPor Favor oremos por...",0
2,"""L' qui mi estudio di 20 anni non realiza lo v...",0
3,##La población Boliviana denuncia los dieferen...,0
4,#AberturasAlCosto.\nDirección Perú esquina Cos...,0
...,...,...
1299,venta,1
1300,Vi esta pilar centro muy grande',1
1301,X favor busco casa o departamento para arquila...,1
1302,"Ya está alquilado?', 'Esta alquilado???'",1


In [7]:
##Crearemos una funcion para limpiar y normalizar el dataset
def cleanAndNorm(dfIn,columnStr):
    
    ##Reemplazaremos los \n por un espacion
    dfIn[columnStr]=dfIn[columnStr].str.replace('\n',' ')
    
    #Lo llevamos a minusculas
    dfIn[columnStr]=dfIn[columnStr].str.lower()
    
    #Normalizamos
    dfIn[columnStr]=dfIn[columnStr].str.normalize('NFKD')
    
    #Encodeamos a ASCII ignorando los que no pueden ser reconocidos
    dfIn[columnStr]=dfIn[columnStr].str.encode('ascii', errors='ignore')
    
    #Hacemos el decode a UTF-8 Estandar
    dfIn[columnStr]=dfIn[columnStr].str.decode('utf-8')
    
    #Eliminamos espacios blancos iniciales y finales
    dfIn[columnStr]=dfIn[columnStr].str.strip() 
    
    #Removemos stopwords
    sw = stopwords.words('spanish')
    #dfIn[columnStr] = dfIn[columnStr].str.replace("|".join(sw), "")
    
    #Removemos los caracteres especiales
    #dfIn[columnStr] = dfIn[columnStr].str.replace("|".join(punctuation), "")
    
    return dfIn

In [8]:
#Limpiamos y normalizamos el dataset con la funcion previamente creada
cldF=cleanAndNorm(df,'Comentario')
cldF

,Comentario,Clasficacion
0,!!del 10 al 20 de septiembre!! disfruta del bo...,0
1,"""ahora si guerra civil""! por favor oremos por ...",0
2,"""l' qui mi estudio di 20 anni non realiza lo v...",0
3,##la poblacion boliviana denuncia los dieferen...,0
4,#aberturasalcosto. direccion peru esquina cost...,0
...,...,...
1299,venta,1
1300,vi esta pilar centro muy grande',1
1301,x favor busco casa o departamento para arquila...,1
1302,"ya esta alquilado?', 'esta alquilado???'",1


In [9]:
#Es hora de separar el conjunto de test y train
X_train, X_test, y_train, y_test = train_test_split(cldF, cldF['Clasficacion'], test_size=0.3, random_state=0, stratify=cldF['Clasficacion'])

In [10]:
#Revisare cuan balanceada me quedo el Split
print(X_train['Clasficacion'].value_counts()) 
print(X_test['Clasficacion'].value_counts()) 

0    712
1    200
Name: Clasficacion, dtype: int64
0    306
1     86
Name: Clasficacion, dtype: int64


In [11]:
# #Creamos el input Train del modelo - TXT
f = open(trainPath, 'w')

for row in X_train.itertuples():
    f.write('__label__'+str(row.Clasficacion)+' '+row.Comentario + "\n")
f.close()

In [12]:
# #Creamos el input Test del modelo - TXT
f = open(testPath, 'w')

for row in X_test.itertuples():
    f.write('__label__'+str(row.Clasficacion)+' '+row.Comentario + "\n")
f.close()

In [13]:
# #Creamos el input del modelo - Lista
# inputModel=[]

# for row in cldF.itertuples():
#     inputModel.append('__label__'+str(row.Clasficacion)+' '+row.Comentario)
    
# inputModel

In [18]:
def print_results(N, p, r):
    print("Number of Examples\t" + str(N))
    print("Precision@{}\t{:.3f}".format(1, p))
    print("Recall@{}\t{:.3f}".format(1, r))

In [65]:
#Entrenamos Modelo
model = train_supervised(
        input=trainPath, lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs'
    )
#lrs=[0.5, 0.1, 0.05], epochs = [10, 25, 50], ngrams = [1,2,3]


In [66]:
##Testeamos Precision
print_results(*model.test(testPath))

Number of Examples	392
Precision@1	0.923
Recall@1	0.923


In [130]:
#Predecimos
texts = ['quiero alquilar una casa en toledo',
         'quiero comprar una casa en pilar',
         'quiero comprar un auto en pilar',
         'busco alquiler en rio segundo',
         'quiero comprar un perro en pilar']

labels = model.predict(texts)
print(labels)

(['__label__0', '__label__1', '__label__0', '__label__1', '__label__1'], array([0.95589507, 0.99988264, 0.69769007, 0.99999994, 0.63759392]))


In [131]:
#Exportamos el modelo para reutulizar a futuro
model.save_model("model_inmobiliara.ftz")